# Практическое задание

## Урок 3. Парсинг данных. HTML, Beautiful Soap

## Вариант 1

In [1]:
# импортируем необходимые библиотеки и модули

from pprint import pprint
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from time import sleep

In [2]:
# создадим списки для параметров вакансий

# наименовние вакансии  
job_vacancy_list = []

# зарплата 
salary_list = []

# минимальная зарплата
min_salary_list = []

# максимальная зарплата 
max_salary_list = []

# валюта зарплаты
currency_salary_list = []

# работодатель 
employers_list = []

# месторасположение
location_list = []

# ссылка на вакансию
link_title_list = []

# сайт вакансии
site_job_title_list = []

In [3]:
# создаем переменную для начальной страницы поиска
num_page = 0

# создаем переменную, содержащую общее количество найденных страниц
num_site_pages = 20

In [4]:
# цикл для каждой страницы поиска 

for page in range(num_site_pages):
    URL_HH_RU = f'https://hh.ru/search/vacancy?text=python&area=3&page={num_page}' 
    USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'

    headers = {
    'User-Agent' : USER_AGENT
    }

    response = requests.get(url=URL_HH_RU, headers=headers)
    dom = bs(response.text, 'html.parser')

    job_vacancy = dom.find('div', {'id': 'a11y-main-content'}) # div с карточками вакансий - родительский 

    # наименовние вакансии 
    for job in job_vacancy:
        job = job.find('a', {'class': 'serp-item__title'})
        if job is not None:
            job_vacancy_list.append(job.text)
        else:
            job_vacancy_list.append(job)

    # зарплата 
    salary = job_vacancy.find('span', {'class': 'bloko-header-section-3'}) 
    for el_sal in job_vacancy:
        el_sal = el_sal.find('span', {'class': 'bloko-header-section-3'})
        if el_sal is None:
            el_sal = 'не указана'
            salary_list.append(el_sal)
        else:
            el_sal = el_sal.text.strip()
            el_sal = el_sal.replace('руб.', '')
            el_sal = el_sal.replace('\u202f', '')
            el_sal = el_sal.replace('от', '')
            el_sal = el_sal.replace('до ', '')
            salary_list.append(el_sal)


    # создаем две копии salary_list для разделения на минимальную и максимальную зарплату
    salary_list_min_prep = salary_list.copy()
    salary_list_max_prep = salary_list.copy()


    # минимальная зарплата
    for index, el in enumerate(salary_list_min_prep, 0):
        if type(el) is not int:
            salary_list_min_prep[index] = el.split(' – ')[0]
            try:
                salary_list_min_prep[index] = int(salary_list_min_prep[index])
            except:
                salary_list_min_prep[index] = salary_list_min_prep[index]

        min_salary_list.append(salary_list_min_prep[index])
    salary_list_min_prep = []


    # максимальная зарплата 
    for index, el in enumerate(salary_list_max_prep, 0):
        if type(el) is not int:
            salary_list_max_prep[index] = el.split(' – ')[-1]
            try:
                salary_list_max_prep[index] = int(salary_list_max_prep[index])
            except:
                salary_list_max_prep[index] = salary_list_max_prep[index]

        max_salary_list.append(salary_list_max_prep[index])
    salary_list_max_prep = []
    salary_list = []

    # работодатель
    for employer in job_vacancy:
        employer = employer.find('a', {'class': 'bloko-link bloko-link_kind-tertiary'})
        if employer is None:
            employer = 'None'
            employers_list.append(employer)
        else:
            employer = employer.text
            employer = employer.strip()
            employer = employer.replace('\xa0', ' ')
            employers_list.append(employer)

    # месторасположение
    for location in job_vacancy:
        location = location.find('div', {'data-qa': 'vacancy-serp__vacancy-address'})
        if location is None:
            emplolocationyer = 'None'
            location_list.append(location)
        else:
            location = location.text
            location = location.strip()
            location = location.replace('\xa0', ' ')
            location_list.append(location)            
            
            
    # ссылка на вакансию
    for link in job_vacancy:
        link = link.find('a', {'class' : 'serp-item__title'})
        if link is None:
            link = "None"
            link_title_list.append(link)
        else:
            link = link['href']
            link_title_list.append(link)
            
    # напечатаем номер страницы, код состояния и ссылку
    print(f'page {num_page} - ok, {response.status_code}, {response.url}')
    
    # увеличим номер страницы на 1 для перехода к следующей
    num_page += 1
    
    sleep(0.5)
            
 # валюта зарплаты и сайт вакансии 

for el in job_vacancy_list:
    cur = 'руб.'
    currency_salary_list.append(cur)
    site = 'https://ekaterinburg.hh.ru/'
    site_job_title_list.append(site)

    
dict_data_job_hh_ru = {
    'наименовние вакансии' : job_vacancy_list,
    'минимальная зарплата' : min_salary_list,
    'максимальная зарплата': max_salary_list,
    'валюта зарплаты': currency_salary_list,
    'ссылка на вакансию': link_title_list,
    'работодатель': employers_list,
    'ссылка на вакансию': link_title_list,
    'работодатель': employers_list,
    'месторасположение' : location_list           
}

dict_data_job_hh_ru_df = pd.DataFrame(dict_data_job_hh_ru)
dict_data_job_hh_ru_df = dict_data_job_hh_ru_df.dropna().reset_index(drop=True)
dict_data_job_hh_ru_df['валюта зарплаты'] = np.where((dict_data_job_hh_ru_df['минимальная зарплата'] == 'не указана'),'не указана', dict_data_job_hh_ru_df['валюта зарплаты'])
dict_data_job_hh_ru_df.head(1)


page 0 - ok, 200, https://ekaterinburg.hh.ru/search/vacancy?text=python&area=3&page=0
page 1 - ok, 200, https://ekaterinburg.hh.ru/search/vacancy?text=python&area=3&page=1
page 2 - ok, 200, https://ekaterinburg.hh.ru/search/vacancy?text=python&area=3&page=2
page 3 - ok, 200, https://ekaterinburg.hh.ru/search/vacancy?text=python&area=3&page=3
page 4 - ok, 200, https://ekaterinburg.hh.ru/search/vacancy?text=python&area=3&page=4
page 5 - ok, 200, https://ekaterinburg.hh.ru/search/vacancy?text=python&area=3&page=5
page 6 - ok, 200, https://ekaterinburg.hh.ru/search/vacancy?text=python&area=3&page=6
page 7 - ok, 200, https://ekaterinburg.hh.ru/search/vacancy?text=python&area=3&page=7
page 8 - ok, 200, https://ekaterinburg.hh.ru/search/vacancy?text=python&area=3&page=8
page 9 - ok, 200, https://ekaterinburg.hh.ru/search/vacancy?text=python&area=3&page=9
page 10 - ok, 200, https://ekaterinburg.hh.ru/search/vacancy?text=python&area=3&page=10
page 11 - ok, 200, https://ekaterinburg.hh.ru/search

,наименовние вакансии,минимальная зарплата,максимальная зарплата,валюта зарплаты,ссылка на вакансию,работодатель,месторасположение
0,Специалист службы поддержки с техническими зна...,15000,39000,руб.,https://ekaterinburg.hh.ru/vacancy/73595106?fr...,Яндекс,Екатеринбург


In [5]:
# сохранение результата в форма "csv"

dict_data_job_hh_ru_df.to_csv('vacancy_parsing_hh_ru.csv', index = False)